In [8]:
!pip show boto3

Name: boto3
Version: 1.26.154
Summary: The AWS SDK for Python
Home-page: https://github.com/boto/boto3
Author: Amazon Web Services
Author-email: 
License: Apache License 2.0
Location: /opt/conda/lib/python3.7/site-packages
Requires: botocore, jmespath, s3transfer
Required-by: sagemaker, sagemaker-studio-analytics-extension, sagemaker-studio-sparkmagic-lib, smclarify


In [9]:
#Basic Functionality 
import pandas as pd
import numpy as np
import math
import sys 
import itertools
import json
import os

#Data Versionin
import dvc

#Carbon Tracking
from codecarbon import track_emissions
from codecarbon import EmissionsTracker
#import codecarbon

#AWS Suggested libraries
import boto3
import pathlib
import io

#SageMaker Specific Libraries
import sagemaker
from sagemaker.deserializers import CSVDeserializer
from sagemaker.serializers import CSVSerializer

from sagemaker.xgboost.estimator import XGBoost
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import (
    ProcessingInput, 
    ProcessingOutput, 
    ScriptProcessor
)
from sagemaker.inputs import TrainingInput

from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import (
    ProcessingStep, 
    TrainingStep, 
    CreateModelStep
)
from sagemaker.workflow.check_job_config import CheckJobConfig
from sagemaker.workflow.parameters import (
    ParameterInteger, 
    ParameterFloat, 
    ParameterString, 
    ParameterBoolean
)
from sagemaker.workflow.clarify_check_step import (
    ModelBiasCheckConfig, 
    ClarifyCheckStep, 
    ModelExplainabilityCheckConfig
)
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet

from sagemaker.workflow.lambda_step import (
    LambdaStep,
    LambdaOutput,
    LambdaOutputTypeEnum,
)
from sagemaker.lambda_helper import Lambda

from sagemaker.model_metrics import (
    MetricsSource, 
    ModelMetrics, 
    FileSource
)
from sagemaker.drift_check_baselines import DriftCheckBaselines

from sagemaker.image_uris import retrieve

#Plot
import matplotlib.pyplot as plt
import seaborn as sns


#Modeling
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

In [10]:
from sagemaker import get_execution_role

role = get_execution_role()
bucket = 'mlops-hw1/hw1'
data_key = 'athletes.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)

df = pd.read_csv(data_location)

In [16]:
!cd mlops2

In [18]:
! git init

Initialized empty Git repository in /root/.git/


In [19]:
!dvc init -f

Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In [23]:
# get data into repo

df.to_csv('mlops2/athletes.csv')

In [25]:
!dvc add mlops2/athletes.csv


⠇[?25l                                                                          ⠋ Collecting stages from the workspace Collecting stages from the workspace Collecting stages from the workspace Collecting stages from the workspace Collecting stages from the workspace Collecting stages from the workspace Collecting stages from the workspace Collecting stages from the workspace Collecting stages from the workspace Collecting stages from the workspace Collecting stages from the workspace Collecting stages from the workspace Collecting stages from the workspace Collecting stages from the workspace Collecting stages from the workspace Collecting stages from the workspace Collecting stages from the workspace Collecting stages from the workspace Checking graph
Adding...                                                                       
!
  0% Checking cache in '/root/.dvc/cache'|           |0/? [00:00<?,    ?files/s]
                                                                         

In [26]:
!git add mlops2/athletes.csv.dvc mlops2/.gitignore

In [27]:
!git commit -m "Add raw data in the original file"

[master (root-commit) 30c689c] Add raw data in the original file
 5 files changed, 11 insertions(+)
 create mode 100644 .dvc/.gitignore
 create mode 100644 .dvc/config
 create mode 100644 .dvcignore
 create mode 100644 mlops2/.gitignore
 create mode 100644 mlops2/athletes.csv.dvc


In [28]:
!dvc remote add -d myremote path
!dvc push

Setting 'myremote' as a default remote.
  0% Transferring|                                   |0/1 [00:00<?,     ?file/s]
!
  0%|          |bcbb43cad35fd37e04ee32b38ce4a6     0.00/? [00:00<?,        ?B/s]
  0%|          |bcbb43cad35fd37e04ee32b38ce4a6 0.00/70.5M [00:00<?,        ?B/s]
 10%|▉         |bcbb43cad35fd37e04ee32b38c7.00M/70.5M [00:00<00:00,    71.1MB/s]
 41%|████      |bcbb43cad35fd37e04ee32b38c29.0M/70.5M [00:00<00:00,     163MB/s]
 52%|█████▏    |bcbb43cad35fd37e04ee32b38c37.0M/70.5M [00:00<00:00,     124MB/s]
1 file pushed                                                                   


In [29]:
!git log --oneline

30c689c (HEAD -> master) Add raw data in the original file


In [ ]:
# go to terminal and git push